<a href="https://colab.research.google.com/github/LEESUSUSUSU/Kaggel/blob/main/1st_place_HMS_inference_code_%ED%95%84%EC%82%AC_%EC%A4%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

#XLA는 기본적으로 모든 사용 가능한 GPU 메모리를 미리 할당(preallocate)하여
#메모리 단편화(fragmentation)를 줄이고 성능을 최적화하려고 합니다.

#기본 메모리 할당자 대신 XLA가 제공하는 메모리 할당기를 사용하는 경우에는 다양한 최적화가 적용될 수 있지만,
#플랫폼의 기본 할당자를 사용하면 더 안정적일 수 있습니다.

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


yamash model

https://arxiv.org/pdf/1905.05702
https://arxiv.org/pdf/1602.02068

Model

I lined up the raw EEG signal as it was and created a single 2D image, which was then processed with a 2D CNN model.
In more detail, 18 signals of bipolar montage were cropped at three different intervals (2000, 5000 and 10000 samples), resized and concatenated into a single image.
I trained several models based on this method. Depending on the model, I changed the range of the bandpass filter to create a 2D image.

This is a very simple method, but the CV was around 0.24.

Following 5 models are used for final sub:

4 x convnext atto models with different seed and bandpass filter (CV: 0.2452, 0.2385, 0.2457, 0.2351)
1 x inception next tiny (CV: 0.2309)


이 코드는 EEG(뇌파) 신호를 2D 이미지로 변환한 후, 2D CNN 모델을 사용하여 처리하는 방법을 설명합니다. 자세히 살펴보면 다음과 같습니다:

EEG 신호 전처리:

원시 EEG 신호를 정렬: 여러 개의 EEG 신호를 정렬하여 단일 2D 이미지를 생성.
신호 크롭: 18개의 bipolar montage 신호를 세 가지 다른 간격(2000, 5000, 10000 샘플)으로 잘라냄.
이미지 크기 조정 및 결합: 크롭된 신호를 크기 조정한 후 단일 이미지로 결합.
모델 학습:

여러 모델을 사용하여 학습 진행.
대역통과 필터: 각 모델에서 대역통과 필터의 범위를 변경하여 2D 이미지를 생성.
모델 성능:

이 방법으로 간단히 구현했지만 교차 검증(CV) 점수는 약 0.24로 나옴.
최종 제출된 모델:
ConvNext Atto 모델 4개: 다른 시드와 대역통과 필터를 사용(CV: 0.2452, 0.2385, 0.2457, 0.2351).
Inception Next Tiny 모델 1개(CV: 0.2309).


In [ ]:
import sys
import os
import gc
import copy
import yaml #YAML형식의 설정 파일을 익고 쓰기 위해 사용됨
import random
import shutil
import time
import typing as tp # 타입 힌팅을 위한 모듈
from glob import glob # 파일 및 디레토리의 경로를 처리하고 파일 패턴 매칭을 위해 사용
from pathlib import Path
from collections import OrderedDict, defaultdict #순서가 있는 딕셔너리와 기본값을 갖는 딕셔너리와 기본값을 갖는 딕셔너리를 제공합니다.
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
# 로깅(logging)기느이을 설정하고 사용
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.signal import butter, lfilter, freqz # 신호 처리 모듈로, Butterworth필터와 필터링 함수를 제공
from sklearn.model_selection import StratifiedGroupKFold

import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler, Adam, AdamW
from torch.cuda import amp
from torch.utils.data import DataLoader, Dataset, default_collate
from torchvision.transforms import v2

import timm # PyTorch 이미지 모델 라이브러리로, 다양한 최신 이미지 모델을 제공합니다.

import albumentations as A # 이미지 증강 라이브러리로, 데이터 증강을 위한 다양한 변환을 제공합니다.
from albumentations.pytorch import ToTensorV2 #ToTensorV2: Albumentations와 PyTorch 간의 호환을 위한 변환기입니다.

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function # __init__ 없음

#init가 없는 이유
#PyTorch의 autograd.Function을 상속받아 정의된 사용자 정의 함수이기 때문입니다
#utograd.Function을 상속받는 클래스는 인스턴스를 생성하여 사용하는 것이 아니라,
#클래스 메소드를 통해 직접 호출되는 방식으로 동작합니다.

#https://course.fast.ai/Lessons/lesson1.html

#EntmaxBisectFunction 클래스 정의
class EntmaxBisectFunction(Function):
  #_gp 클래스 메소드 정의: x를 alpha-1 제곱한 값을 반환
  # x를 ålph-1  제곱한 값을 반환 하는 클래스 메소드
    @classmethod
    def _gp(cls, x, alpha):
        return x ** (alpha - 1)


    # _gp_inv 클래스 메소드 정의: y를 1/(alpha-1) 제곱한 값을 반환
    @classmethod
    def _gp_inv(cls, y, alpha):
        return y ** (1 / (alpha - 1))

    # _p 클래스 메소드 정의: X를 0 이상으로 클램프한 후, _gp_inv를 적용한 값을 반환
    @classmethod
    def _p(cls, X, alpha):
        return cls._gp_inv(torch.clamp(X, min=0), alpha)

    # forward 클래스 메소드 정의
    # 순전파 게산을 위한 클래스 메소드

    #순전파는 신경망의 입력이 주어졌을 때, 이를 출력으로 변환하는 과정을 의미합니다.
    #주어진 코드에서는 데이터 전처리 및 모델 학습을 위한 데이터 로더를 준비하는 부분이 주를 이루고 있으며,
    #직접적으로 모델의 순전파를 수행하는 부분은 포함되어 있지 않습니다.

    @classmethod
    def forward(cls, ctx, X, alpha=1.5, dim=-1, n_iter=50, ensure_sum_one=True):
       # alpha가 torch.Tensor 타입이 아니면, torch.Tensor로 변환
        if not isinstance(alpha, torch.Tensor):
            alpha = torch.tensor(alpha, dtype=X.dtype, device=X.device)
        # alpha의 shape을 X와 일치하도록 확장
        alpha_shape = list(X.shape)
        alpha_shape[dim] = 1
        alpha = alpha.expand(*alpha_shape)

        # 컨텍스트에 alpha와 dim 저장 (역전파 시 사용)
        ctx.alpha = alpha
        ctx.dim = dim
        d = X.shape[dim]

       # X의 해당 차원에서 최대값 계산
        max_val, _ = X.max(dim=dim, keepdim=True)
        X = X * (alpha - 1)
        max_val = max_val * (alpha - 1)

        ## tau_lo와 tau_hi 초기값 설정
        # Note: when alpha < 1, tau_lo > tau_hi. This still works since dm < 0.

        tau_lo = max_val - cls._gp(1, alpha)
        tau_hi = max_val - cls._gp(1 / d, alpha)

        # Note: f_lo should always be non-negative.
        # f_lo = cls._p(X - tau_lo, alpha).sum(dim) - 1
        # tau_lo와 tau_hi의 차이
        dm = tau_hi - tau_lo


        # # 이분법적 접근을 통해 최적의 tau 값을 찾기 위해 반복
        for it in range(n_iter):

            dm /= 2
            tau_m = tau_lo + dm
            p_m = cls._p(X - tau_m, alpha)
            f_m = p_m.sum(dim) - 1
        # f_m이 0 이상인 경우 tau_lo를 tau_m으로 업데이트
            mask = (f_m >= 0).unsqueeze(dim)
            tau_lo = torch.where(mask, tau_m, tau_lo)
        # ensure_sum_one 옵션이 True일 경우, p_m을 정규화하여 합이 1이 되도록 함
        if ensure_sum_one:
            p_m /= p_m.sum(dim=dim).unsqueeze(dim=dim)

        # 컨텍스트에 p_m 저장 (역전파 시 사용)
        ctx.save_for_backward(p_m)

        return p_m
    # 역전파 계산을 위한 클래스 메소드
    @classmethod
    def backward(cls, ctx, dY):


      # 순전파에서 저장한 p_m을 불러옴
        Y, = ctx.saved_tensors
        # Y가 0보다 큰 경우, Y ** (2 - alpha)를 계산
        gppr = torch.where(Y > 0, Y ** (2 - ctx.alpha), Y.new_zeros(1))

        # dY와 gppr을 곱하여 dX 계산
        dX = dY * gppr
        q = dX.sum(ctx.dim) / gppr.sum(ctx.dim)
        q = q.unsqueeze(ctx.dim)
        dX -= q * gppr
        # alpha에 대한 그라디언트 초기화
        d_alpha = None
        if ctx.needs_input_grad[1]:

            # alpha gradient computation
            # d_alpha = (partial_y / partial_alpha) * dY
            # NOTE: ensure alpha is not close to 1
            # since there is an indetermination
            # batch_size, _ = dY.shape

            # shannon terms
            S = torch.where(Y > 0, Y * torch.log(Y), Y.new_zeros(1))
            # shannon entropy
            ent = S.sum(ctx.dim).unsqueeze(ctx.dim)
            Y_skewed = gppr / gppr.sum(ctx.dim).unsqueeze(ctx.dim)
            # alpha에 대한 그라디언트 계산
            d_alpha = dY * (Y - Y_skewed) / ((ctx.alpha - 1) ** 2)
            d_alpha -= dY * (S - Y_skewed * ent) / (ctx.alpha - 1)
            d_alpha = d_alpha.sum(ctx.dim).unsqueeze(ctx.dim)

        return dX, d_alpha, None, None, None

# entmax_bisect 함수를 정의하여 EntmaxBisectFunction을 간편하게 사용할 수 있도록 함
def entmax_bisect(X, alpha=1.5, dim=-1, n_iter=50, ensure_sum_one=True):
    return EntmaxBisectFunction.apply(X, alpha, dim, n_iter, ensure_sum_one)

딥러닝 모델에서 순전파(Forward Pass)와 역전파(Backward Pass)는 모델의 학습과 추론에 필수적인 과정입니다. 이와 함께, 주어진 코드에서는 이분법적 접근법이 사용되고 있는데, 이러한 이유에 대해 자세히 설명드리겠습니다.

순전파 (Forward Pass)
순전파는 입력 데이터를 모델에 통과시켜 출력을 얻는 과정입니다. 주어진 데이터를 모델의 각 층을 통과시키면서 계산을 수행하고, 최종 출력을 도출합니다. 순전파의 목적은 다음과 같습니다:

예측: 입력 데이터에 대한 예측 값을 생성합니다.
손실 계산: 예측 값과 실제 값 사이의 차이를 손실 함수(Loss Function)를 통해 계산합니다.
역전파 (Backward Pass)
역전파는 순전파에서 계산된 손실을 기반으로, 모델의 가중치를 업데이트하기 위해 각 층의 그라디언트를 계산하는 과정입니다. 역전파의 목적은 다음과 같습니다:

그라디언트 계산: 손실 함수의 출력에 대한 각 가중치의 기울기(그라디언트)를 계산합니다.
가중치 업데이트: 옵티마이저(Optimizer)를 사용하여 그라디언트를 기반으로 가중치를 업데이트합니다. 이를 통해 모델이 점차적으로 학습합니다.
역전파는 체인 룰(Chain Rule)을 사용하여 계산되며, 이는 모델의 각 층을 역순으로 따라가며 그라디언트를 계산합니다.

이분법적 접근 (Bisection Method)
이분법적 접근은 주어진 범위 내에서 특정 조건을 만족하는 값을 찾기 위한 방법입니다. 주어진 코드에서는 주로 EEG 데이터를 처리할 때 이분법적 접근이 사용됩니다. 이 방법의 목적과 이유는 다음과 같습니다:

효율적인 최적화: 이분법적 접근은 반복적으로 범위를 절반으로 줄여가며 최적의 값을 찾기 때문에, 빠르고 효율적으로 최적화 문제를 해결할 수 있습니다.
정확한 값 찾기: 특정 조건을 만족하는 값을 정확하게 찾을 수 있으며, 이는 주로 수렴 조건을 만족하는 최적의 파라미터를 찾기 위해 사용됩니다.
EEG 데이터 처리: 주어진 코드에서는 EEG 데이터에서 특정 주파수 대역을 추출하거나, 특정 조건을 만족하는 신호를 찾기 위해 이분법적 접근을 사용합니다.
코드에서의 순전파, 역전파, 이분법적 접근
순전파:

모델의 입력 데이터를 통과시켜 예측 값을 생성하는 과정입니다.
코드 예시: DataGenerator 클래스에서 데이터를 전처리하고 모델의 입력 형식으로 준비하는 부분.
역전파:

손실 함수의 결과를 기반으로 모델의 가중치를 업데이트하는 과정입니다.
이 코드에서는 직접적인 역전파 구현이 없지만, 일반적으로 TensorFlow 또는 PyTorch의 fit 또는 train 메소드에서 자동으로 수행됩니다.
이분법적 접근:

특정 조건을 만족하는 최적의 값을 찾기 위해 반복적으로 범위를 절반으로 줄여가며 최적의 값을 찾는 방법입니다.
코드 예시: EntmaxBisectFunction 클래스에서 순전파 과정에서 최적의 tau 값을 찾기 위해 이분법적 접근을 사용합니다.
결론
순전파와 역전파는 딥러닝 모델 학습의 기본적인 과정으로, 모델이 입력 데이터를 처리하고 가중치를 학습하는 데 사용됩니다. 이분법적 접근은 효율적으로 최적의 값을 찾기 위한 방법으로, 특정 문제에서 빠르고 정확하게 해를 찾는 데 유용합니다. 이러한 방법들은 모델의 성능을 최적화하고, 주어진 문제를 효과적으로 해결하기 위해 사용됩니다.